In [22]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB
import numpy as np

import sys
sys.path.append('../../')
import saedfsc

In [23]:
numCustomersTypes = 25
numSubsystems = 10
numObjectives = 11

# Create a list of supplier names
names = ['CustomerType' + str(i) for i in range(1, numCustomersTypes + 1)]

qty = np.random.randint(low=25, high=1000, size=numCustomersTypes)
priceFocus = np.random.random(size=numCustomersTypes)

partworthWts = [np.array2string(np.random.random(numSubsystems), separator=',') for _ in range(numCustomersTypes)]
perfWts = [np.array2string(np.random.random(numObjectives), separator=',') for _ in range(numCustomersTypes)]

# Create the DataFrame
df = DataFrame({
    'Name': names,
    'Quantity': qty,
    'PriceFocus': priceFocus,
    'PartworthUtilityWeights': partworthWts,
    'PerformanceUtilityWeights': perfWts
})
df.to_csv('../resources/customers.csv', index=False)
customers: DataFrame = read_csv('../resources/customers.csv')
customers

,Name,Quantity,PriceFocus,PartworthUtilityWeights,PerformanceUtilityWeights
0,CustomerType1,469,0.391040,"[0.91450978,0.08733943,0.51950344,0.55561313,0...","[0.34496792,0.6898727 ,0.13100207,0.50656613,0..."
1,CustomerType2,929,0.951341,"[0.66601538,0.54644132,0.24889524,0.86383297,0...","[0.46336316,0.47837937,0.80161025,0.20513706,0..."
2,CustomerType3,294,0.740209,"[0.52786049,0.73410731,0.85361372,0.83415508,0...","[0.92694124,0.48651615,0.78058322,0.45806116,0..."
3,CustomerType4,834,0.608726,"[0.30815853,0.44577241,0.59348252,0.60900538,0...","[0.45334148,0.17085646,0.67141338,0.78225234,0..."
4,CustomerType5,733,0.565038,"[0.7610932 ,0.05656771,0.61822428,0.8374061 ,0...","[0.64583732,0.09631073,0.23214996,0.88206262,0..."
5,CustomerType6,35,0.254575,"[0.5931458 ,0.67793322,0.22446516,0.4948187 ,0...","[0.57868043,0.07927113,0.05214455,0.78084477,0..."
6,CustomerType7,239,0.362928,"[0.66483611,0.90905972,0.18731687,0.66342118,0...","[0.54702712,0.05151569,0.09804964,0.04074776,0..."
7,CustomerType8,564,0.770436,"[0.76932255,0.36910923,0.89955738,0.1623181 ,0...","[0.56254937,0.13950323,0.18176402,0.34446781,0..."
8,CustomerType9,927,0.599208,"[0.51947281,0.1023688 ,0.59192487,0.60865134,0...","[0.53924253,0.90808112,0.34176956,0.47190584,0..."
9,CustomerType10,366,0.452926,"[0.09527526,0.13840712,0.67149965,0.23045829,0...","[0.09699416,0.73592342,0.92129521,0.89635862,0..."


Print dictionary of customers and their weights.

In [24]:
name_weights_dict = customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWts = {c : np.fromstring(name_weights_dict[c].strip('[]'), sep=',') for c in name_weights_dict}
cWts

{'CustomerType1': array([0.34496792, 0.6898727 , 0.13100207, 0.50656613, 0.56858667,
        0.41068758, 0.77968098, 0.73766977, 0.31568117, 0.71535052,
        0.7180232 ]),
 'CustomerType2': array([0.46336316, 0.47837937, 0.80161025, 0.20513706, 0.94625555,
        0.9862123 , 0.91018534, 0.29387321, 0.27523141, 0.12271913,
        0.53413563]),
 'CustomerType3': array([0.92694124, 0.48651615, 0.78058322, 0.45806116, 0.66226228,
        0.90029841, 0.65342589, 0.87910133, 0.95065054, 0.82361401,
        0.64281367]),
 'CustomerType4': array([0.45334148, 0.17085646, 0.67141338, 0.78225234, 0.52737089,
        0.56592381, 0.30855409, 0.64644858, 0.77395197, 0.45585546,
        0.56333843]),
 'CustomerType5': array([0.64583732, 0.09631073, 0.23214996, 0.88206262, 0.24797481,
        0.87841117, 0.32040573, 0.1439905 , 0.05921197, 0.4861102 ,
        0.3764227 ]),
 'CustomerType6': array([0.57868043, 0.07927113, 0.05214455, 0.78084477, 0.66872147,
        0.95035967, 0.37018668, 0.901857

In [25]:
cQty = dict(zip(customers['Name'], customers['Quantity'])) # customer quantities
cQty

{'CustomerType1': 469,
 'CustomerType2': 929,
 'CustomerType3': 294,
 'CustomerType4': 834,
 'CustomerType5': 733,
 'CustomerType6': 35,
 'CustomerType7': 239,
 'CustomerType8': 564,
 'CustomerType9': 927,
 'CustomerType10': 366,
 'CustomerType11': 286,
 'CustomerType12': 372,
 'CustomerType13': 143,
 'CustomerType14': 660,
 'CustomerType15': 201,
 'CustomerType16': 240,
 'CustomerType17': 383,
 'CustomerType18': 799,
 'CustomerType19': 752,
 'CustomerType20': 435,
 'CustomerType21': 540,
 'CustomerType22': 860,
 'CustomerType23': 874,
 'CustomerType24': 561,
 'CustomerType25': 448}